In [2]:
import psycopg2

In [31]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# the average duration of the trip per route 
sql = '''
select
    route_id, 
    stop_id_start, 
    stop_id_end,
    to_char(max(end_stop_time - init_stop_time), 'HH24:MI:SS') as diff_time
from
    warehouse
group by cube (route_id, stop_id_start, stop_id_end)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    if row[1] == None and row[2] == None:
        print(f'route {row[0]}, max_time {row[-1]}h')

In [28]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# the number of trips at each time per route
sql = '''
select
    route_id, 
    to_char(init_stop_time, 'HH24:MI:SS') as init_time,
    count(*)
from
    warehouse 
group by cube (route_id, init_stop_time)
order by count(*) desc
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    print(f'route id {row[0]}, init stop time {row[1]}, amount [')

(None, None, 1434319)
('207', None, 108461)
('500', None, 68674)
('206', None, 67052)
('403', None, 62229)


In [29]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# per trip_id, the delay (how much time the bus arrives after the taxi), 
# the taxi trip duration, and the bus trip duration
sql = '''
select
    trip_id,
    to_char(initial_ts, 'HH24:MI:SS') as i_ts,
    to_char(init_stop_time, 'HH24:MI:SS') as i_stop_time,
    to_char(max(init_stop_time - initial_ts), 'HH24:MI:SS') as diff_time,
    to_char(max(final_ts - initial_ts), 'HH24:MI:SS') as taxi_duration,
    to_char(max(end_stop_time - init_stop_time), 'HH24:MI:SS') as bus_duration
from
    warehouse 
group by cube (trip_id, initial_ts, init_stop_time)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    print(f'trip_id {row[0]}, delay {row[3]}h, taxi duration {row[4]}h, bus duration {row[5]}h')

trip_id 106_0_U_10, delay 00:10:53h, taxi duration 00:05:34h, bus duration 00:04:36h
trip_id 106_0_U_10, delay 00:10:53h, taxi duration 00:05:34h, bus duration 00:04:36h
trip_id 106_0_U_10, delay 00:10:53h, taxi duration 00:05:34h, bus duration 00:04:36h
trip_id 106_0_U_14, delay 00:27:46h, taxi duration 00:10:27h, bus duration 00:04:35h
trip_id 106_0_U_14, delay 00:27:46h, taxi duration 00:10:27h, bus duration 00:04:35h


In [30]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# per trip_id, the bus delay, and trip delay
sql = '''
select
    trip_id,
    to_char(max(init_stop_time - initial_ts), 'HH24:MI:SS') as bus_delay,
    to_char(max(end_stop_time - final_ts), 'HH24:MI:SS') as arrival_delay
from
    warehouse 
group by cube (trip_id, initial_ts, init_stop_time)
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
for row in results[:5]:
    print(f'trip id {row[0]}, bus delay {row[1]}h, arrival delay {row[2]}h')

trip id 106_0_U_10, bus delay 00:10:53h, arrival delay 00:09:55h
trip id 106_0_U_10, bus delay 00:10:53h, arrival delay 00:09:55h
trip id 106_0_U_10, bus delay 00:10:53h, arrival delay 00:09:55h
trip id 106_0_U_14, bus delay 00:27:46h, arrival delay 00:21:54h
trip id 106_0_U_14, bus delay 00:27:46h, arrival delay 00:21:54h
